In [1]:
#Importing the required libraries

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from PIL import Image # converting images into arrays

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

from wordcloud import WordCloud, STOPWORDS


Libraries imported.


In [2]:
address = 'Tempe, Arizona'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

/home/prakash/PREFIX=/home/codebind/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto, Canada are 33.4255056, -111.9400125.


In [3]:
!wget "https://www.downloadexcelfiles.com/sites/default/files/docs/list-cities-arizona-state-us-166j.csv" 

--2020-06-08 16:57:18--  https://www.downloadexcelfiles.com/sites/default/files/docs/list-cities-arizona-state-us-166j.csv
Resolving www.downloadexcelfiles.com (www.downloadexcelfiles.com)... 172.104.160.11, 2400:8901::f03c:91ff:feb3:9443
Connecting to www.downloadexcelfiles.com (www.downloadexcelfiles.com)|172.104.160.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6492 (6.3K) [text/csv]
Saving to: ‘list-cities-arizona-state-us-166j.csv’

list-cities-arizona 100%[===================>]   6.34K  --.-KB/s    in 0s      

2020-06-08 16:57:19 (117 MB/s) - ‘list-cities-arizona-state-us-166j.csv’ saved [6492/6492]



In [69]:
arizona_data = pd.read_csv('list-cities-arizona-state-us-166j.csv', encoding = 'latin1')

In [70]:
arizona_data = arizona_data.drop(['Incorporated', 'Type'], axis=1)


In [71]:
arizona_data = arizona_data.rename({"Name": "Neighborhood", "County" : "Borough"}, axis=1)

In [84]:
arizona_data

,Neighborhood,Borough,Population,Area (2010),Density (2010)
0,NaN,NaN,-2010,NaN,(Pop./mi2)
1,Apache Junction,Pinal,"35,840",35.00 sq mi (90.6 km2),"1,024.20"
2,Avondale,Maricopa,"76,238",45.65 sq mi (118.2 km2),"1,672.00"
3,Benson,Cochise,"5,105",41.46 sq mi (107.4 km2),123.3
4,Bisbee,Cochise,"5,575",5.16 sq mi (13.4 km2),"1,080.30"
5,Buckeye,Maricopa,"8,173",375.39 sq mi (972.3 km2),135.6
6,Bullhead City,Mohave,"39,540",60.18 sq mi (155.9 km2),665.9
7,Camp Verde,Yavapai,"10,873",43.15 sq mi (111.8 km2),252
8,Carefree,Maricopa,"3,363",8.81 sq mi (22.8 km2),382.2
9,Casa Grande,Pinal,"48,571",109.67 sq mi (284.0 km2),442.9


In [72]:
arizona_data.columns

Index(['Neighborhood', 'Borough', 'Population', 'Area (2010)',
       'Density (2010)'],
      dtype='object')

In [73]:
ar = pd.DataFrame(arizona_data, columns = ['Borough', 'Neighborhood'])
ar.shape

(96, 2)

In [74]:
ar.dropna(inplace = True)


In [75]:
ar.shape

(91, 2)

In [76]:
ar.columns

Index(['Borough', 'Neighborhood'], dtype='object')

In [77]:
ar.sort_values('Borough')

,Borough,Neighborhood
72,Apache,Springerville
21,Apache,Eagar
73,Apache,St. Johns
68,Cochise,Sierra Vista
81,Cochise,Tombstone
86,Cochise,Willcox
36,Cochise,Huachuca City
4,Cochise,Bisbee
3,Cochise,Benson
19,Cochise,Douglas


In [78]:
for borough, neighborhood in zip(ar['Borough'],ar['Neighborhood']):   
    #find the location data, ignore the neighborhoods that are unable to be located by Nominatim
    geolocator = Nominatim(user_agent = 'arizona')
    location = geolocator.geocode("{},{},Arizona".format(neighborhood,borough))
    
    #try one more searching
    if location is None: 
        location = geolocator.geocode("{},Arizona".format(neighborhood))
        
    if location is None: 
        print("The location data of {} in {} is not available!".format(neighborhood,borough))
    else:
        ar = ar.append({
                                                'Borough': borough,
                                                'Neighborhood': neighborhood,
                                                'Latitude': location.latitude,
                                                'Longitude': location.longitude
                                               }, ignore_index=True)

ar.head()

,Borough,Neighborhood,Latitude,Longitude
0,Pinal,Apache Junction,NaN,NaN
1,Maricopa,Avondale,NaN,NaN
2,Cochise,Benson,NaN,NaN
3,Cochise,Bisbee,NaN,NaN
4,Maricopa,Buckeye,NaN,NaN


In [91]:
ar = ar.dropna()

In [109]:
#Use Foursquare to expore the neighborhoods
#My Foursquare Credentials 
CLIENT_ID = 'S1KVEF2AKPAYLU1ETMETRFO3R0FWOUJXWWGQ55TILVFOAOLY' 
CLIENT_SECRET = 'G1EAHHBTQRSOP4JVOIQBM4QV452XF5JZBLV04MV0CSPKJLAF' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # only return the top 100 venues

# define a function to expore each city in radius of 1000 meters
def getNearbyVenues(nborhood, radius=1000):
    
    venues=pd.DataFrame(columns=['Borough','Neighborhood','VenueName','VenueCategory', 'VenueLatitude', 'VenueLongitude'])
    for borough, neighborhood, lat, lng in zip(nborhood['Borough'], nborhood['Neighborhood'], nborhood['Latitude'], nborhood['Longitude']):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        if results is not None:
            for v in results:
                venues = venues.append({'Borough':borough, 'Neighborhood': neighborhood, 
                           'VenueName': v['venue']['name'], 
                           'VenueCategory': v['venue']['categories'][0]['name'],
                           'VenueLatitude' : v['venue']['location']['lat'],
                           'VenueLongitude' : v['venue']['location']['lng']},ignore_index=True)

   
    
    return(venues)

In [100]:
def citymap(cityname,countryname,dataframe):
    # create map
    address = cityname + ',' + countryname

    geolocator = Nominatim(user_agent="arizona")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    my_map = folium.Map(location=[latitude, longitude], zoom_start=8)

    # set color scheme for the Borough
    borough_name = dataframe['Borough'].unique().tolist()
    colnum = dataframe['Borough'].unique().size
    x = np.arange(colnum)
    ys = [i+x+(i*x)**2 for i in range(colnum)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    
    for lat, lon, neighborhood, borough in zip(dataframe['Latitude'], dataframe['Longitude'], dataframe['Neighborhood'], dataframe['Borough']):
        cluster = borough_name.index(borough)
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster],
            fill_opacity=0.7).add_to(my_map)
       
    
    return my_map


In [101]:
citymap('Tempe','United States',ar)

In [110]:
ar_venues = getNearbyVenues(ar)
ar_venues.head()

,Borough,Neighborhood,VenueName,VenueCategory,VenueLatitude,VenueLongitude
0,Pinal,Apache Junction,Handlebar Pub And Grill,Bar,33.415463,-111.553158
1,Pinal,Apache Junction,CVS pharmacy,Pharmacy,33.413972,-111.549677
2,Pinal,Apache Junction,Los Gringos Locos,Mexican Restaurant,33.412419,-111.551119
3,Pinal,Apache Junction,The Good Apple,Farmers Market,33.416737,-111.550773
4,Pinal,Apache Junction,Walgreens,Pharmacy,33.412688,-111.547010


In [117]:
ar_venues

,Borough,Neighborhood,VenueName,VenueCategory,VenueLatitude,VenueLongitude
0,Pinal,Apache Junction,Handlebar Pub And Grill,Bar,33.415463,-111.553158
1,Pinal,Apache Junction,CVS pharmacy,Pharmacy,33.413972,-111.549677
2,Pinal,Apache Junction,Los Gringos Locos,Mexican Restaurant,33.412419,-111.551119
3,Pinal,Apache Junction,The Good Apple,Farmers Market,33.416737,-111.550773
4,Pinal,Apache Junction,Walgreens,Pharmacy,33.412688,-111.547010
5,Pinal,Apache Junction,DirtWater Springs,American Restaurant,33.415521,-111.552428
6,Pinal,Apache Junction,Hackers Grill,American Restaurant,33.417804,-111.550808
7,Pinal,Apache Junction,Dollar General,Discount Store,33.413334,-111.554191
8,Pinal,Apache Junction,Fry's Food Store,Grocery Store,33.413856,-111.543671
9,Pinal,Apache Junction,Starbucks,Coffee Shop,33.412258,-111.549213


In [122]:
borough_name = ar_venues['Borough'].unique().tolist()
colnum = ar_venues['Borough'].unique().size
x = np.arange(colnum)
ys = [i+x+(i*x)**2 for i in range(colnum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [130]:
# create map of Scarborough using latitude and longitude values
map_venues = folium.Map(location=[latitude, longitude], zoom_start=16)

# add markers to map
for lat, lon, Venue, Neighborhood in zip(ar_venues['VenueLatitude'], ar_venues['VenueLongitude'], ar_venues['VenueName'],  ar_venues['Neighborhood']):
    cluster = borough_name.index(borough)
    label = '{}, {}'.format(Venue, Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color= rainbow[cluster],
        fill=True,
        fill_color= rainbow[cluster],
        fill_opacity=0.4,
        parse_html=False).add_to(map_venues)  
    
map_venues